In [2]:
!pip install datasets
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 80.1 MB/s eta 0:00

In [3]:
from datasets import get_dataset_split_names, get_dataset_config_names, \
    load_dataset
from transformers import AutoTokenizer
from typing import List
from tqdm import tqdm
from statistics import pstdev, mean

In [4]:
class XNLIAnalyzer:
    def __init__(self):
        self.dataset_name = 'xnli'
        self.dataset = dict()
        self.features = [['premise', 'hypothesis'], 'label']
        self.languages = self.get_languages()
        self.split_names = get_dataset_split_names(self.dataset_name, 'all_languages')

    def get_languages(self):
        config_names = get_dataset_config_names(self.dataset_name)
        languages = [x for x in config_names if len(x) == 2]
        return languages

    def count_max_token(self, lang: str, tokenizer: AutoTokenizer) -> List[int]:
        tokens_length = dict()
        for i, split in enumerate(self.split_names):
            tokens = []
            # feats = [(x['premise'], x['hypothesis']) for x in self.dataset[lang][split]]
            for line in tqdm(self.dataset[lang][split]):
                item = (line['premise'], line['hypothesis'])
                inputs = tokenizer.encode(item, return_tensors="pt")
                tokens.append(len(inputs[0]))
            tmax = max(tokens)
            tmin = min(tokens)
            tmean = mean(tokens)
            stdev = pstdev(tokens)
            tokens_length[split] = {'max': tmax, 'min': tmin, 'mean': tmean, 'stdev': stdev}
            print(f'{lang}-{split}: {tmax} - {tmin} - {tmean} - {stdev}')
        return tokens_length

    def get_max_length(self, checkpoint: str) -> dict:
        tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        results = {'tokenizer': checkpoint}
        for lang in self.languages:
            self.dataset[lang] = load_dataset(self.dataset_name, lang)
            results[lang] = self.count_max_token(lang, tokenizer=tokenizer)
        return results

In [5]:
xnli = XNLIAnalyzer()
checkpoints = ['bigscience/bloomz-560m', 'bigscience/bloomz-3b']
res = []
for i, check in enumerate(checkpoints):
    res.append(xnli.get_max_length(check))
    print(check)
    print(res[i])

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5010 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Dataset xnli downloaded and prepared to /root/.cache/huggingface/datasets/xnli/ar/1.1.0/818164464f9c9fd15776ca8a00423b074344c3e929d00a2c1a84aa5a50c928bd. Subsequent calls will reuse this data.


...

zh-train: 470 - 2 - 45.51596121231876 - 23.76952946260383


100%|██████████| 5010/5010 [00:01<00:00, 3581.86it/s]


zh-test: 77 - 8 - 28.10059880239521 - 10.471054087021606


100%|██████████| 2490/2490 [00:00<00:00, 3586.71it/s]


zh-validation: 69 - 5 - 27.842168674698794 - 10.364423477054066
bigscience/bloomz-3b
{'tokenizer': 'bigscience/bloomz-3b', 'ar': {'train': {'max': 364, 'min': 2, 'mean': 38.96246263069707, 'stdev': 19.790556012438046}, 'test': {'max': 106, 'min': 8, 'mean': 36.34970059880239, 'stdev': 13.476595551357542}, 'validation': {'max': 81, 'min': 7, 'mean': 35.791967871485944, 'stdev': 13.071386319139553}}, 'bg': {'train': {'max': 781, 'min': 4, 'mean': 81.7038797867085, 'stdev': 44.48071982281658}, 'test': {'max': 194, 'min': 14, 'mean': 81.79041916167665, 'stdev': 30.848014329091853}, 'validation': {'max': 226, 'min': 15, 'mean': 80.92329317269076, 'stdev': 29.801258355928205}}, 'de': {'train': {'max': 592, 'min': 2, 'mean': 57.345771093602785, 'stdev': 30.320349270668505}, 'test': {'max': 129, 'min': 10, 'mean': 54.02075848303393, 'stdev': 19.536014345575502}, 'validation': {'max': 114, 'min': 13, 'mean': 53.75903614457831, 'stdev': 18.89080248025719}}, 'el': {'train': {'max': 1182, 'min': 2